Pytorch Autograd
================

If you flag a torch Tensor with the attribute `x.requires_grad=True`, then pytorch will automatically keep track the computational history of all tensors that are derived from `x`.  This allows pytorch to figure out derivatives of any scalar result with regard to changes in the components of x.

The function `torch.autograd.grad(output_scalar, [list of input_tensors])` computes `d(output_scalar)/d(input_tensor)` for each input tensor component in the list.  For example:

In [ ]:
import torch
from matplotlib import pyplot as plt

x = torch.linspace(0, 5, 100, requires_grad=True)
y = (x**2).sin()
dydx = torch.autograd.grad(y.sum(), [x])[0]

plt.plot(x.detach(), y.detach(), label='y')
plt.plot(x.detach(), dydx, label='dy/dx')
plt.legend()
plt.show()

(Note that in the example above, because the components of the vector space are independent of each other, we happen to have `d(y.sum())/dx[i] = dy[i]/dx[i]`, so computing a single gradient vector of the sum is equiavlent to computing componentwise derivatives.)

What is `x.detach()` and `y.detach()`?  Every tensor that depends on `x` will also be `requires_grad=True`, so both `x` and `y` are requires-grad tensors.  Pytorch prevents requires-grad tensors from being converted to numpy or regular numbers (because pytorch cannot track dependencies outside of pytorch tensors).  To get the data in a requires-grad tensor, you neeed to first explicitly get a detached reference to the data using `x.detach()`.

Backprop and In-place gradients
-------------------------------

In a typical nerual network we will not just be getting gradients with regard to one input like `x` above, but with regard to a list of dozens or hundreds of tensor parameters that have all been marked with `requires_grad=True`.  It can be inconvenient to keep track of which gradient outputs go with which original tensor input.  But since the gradients have exactly the same shape as the inputs, it is natural to store computed gradients in-place on the tensors themselves.

To simplify this common operation, pytorch provides the `y.backward()` method, which computes the gradients of y with respect to *every* requires-grad dependency, and stores the results in the field `x.grad` for every original input vector `x` that was marked as `requires_grad=True`.

In [ ]:
x = torch.linspace(0, 5, 100, requires_grad=True)
y = (x**2).sin()
y.sum().backward()   # populates the grad attribute below.
print(x.grad)

plt.plot(x.detach(), y.detach(), label='y')
plt.plot(x.detach(), x.grad, label='dy/dx')
plt.legend()
plt.show()

Accumulating and Zeroing grad
-----------------------------

**Gradient accumulation.** If you find that your data batches are too large to get gradients of the
whole thing, then it is usually possible to split the batches into smaller pieces and add the
gradients. Because gradient accumulation is a common pattern, if you call `.backward()` when parameters
`x.grad` already exists, it is not an error.  The new gradient will be *added* to the old one.

**zero_grad().** This means that you need to set any previous value of `x.grad` to zero before
running `backward()`, or else the new gradient will be added to the old one.  Optimizers have a
utility `optim.zero_grad()` to do this to all the optimized parameters at once.

Saving memory on inference
--------------------------

**Avoiding autograd when you don't need it.** Normally, all the parameters of a neural network are set to `requires_grad=True` by default, so they are ready to be trained.  But that means that whenever you run a network, you will get output which is also requires-grad, and it will be attached to a long computation history that consumes a lot of precious GPU memory.

To avoid all this expense when you have no intention of training the network, you could go through all the network parameters to set `requires_grad=False`.

Another way to avoid the computation history is to enclose the entire computation within a `with torch.no_grad():` block.  This will suppress all the autograd mechanics (which means, of course, `.backward()` will not function).

Note that this is different from the role of `net.eval()` which puts puts the network in inference mode computationally (batchnorm, dropout, and other operations behave differently in training and inference); `net.eval()` does not have any effect on `requires_grad`.

More tricks
-----------

**Gradients over intermediate values.** Normally gradients with respect to intermediate values are not stored in `.grad` - just original input variables - but you can ask for intermediate gradients to be stored using `v.retain_grad()`.

**Second derivatives.** If you want higher-order derivatives, then you want pytorch to build the computation graph when it is computing the gradient itself, so this graph can be differentiated again.  To do this, use the `create_graph=True` option on the `grad` or `backward` methods.

**Gradients of more than one objective.** Usually you can call `y.backward()` only once - to save memory, pytorch will deallocate the computation graph after you have computed a single gradient.  But if you need more than one gradient (e.g., if you have different objectives that you want to apply to different parameters, as with happens with GANs sometmies), you can use `retain_graph=True`.
